In [1]:
#increase the display size to 95% of window width
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

import os
import shutil
import subprocess
import gc
import xarray as xr
import numpy as np
import pymp

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.transforms import Bbox
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

from functions.mandyocIO import read_mandyoc_output, read_datasets, change_dataset, _calc_melt_dry, _calc_melt_wet

path = os.getcwd().split('/') # Get local file
machine_path = '/'+path[1]+'/'+path[2] # Select home according to OS.

# Estimating Melt volume

Gerya, T. (2019). Introduction to numerical geodynamic modelling. Cambridge University Press. 2nd edition, Chapter 21. The melt fraction $M$ is given by:

$$M = 0 \text{ at } T < T_{\mathrm{liquidus}}\mathrm{,}$$

$$M = \frac{T - T_{\mathrm{solidus}}}{T_{\mathrm{liquidus}} - T_{\mathrm{solidus}}} \text{ at } T_{\mathrm{solidus}} T < T_{\mathrm{liquidus}} \mathrm{,}$$

$$M = 1 \text{ at } T \geq T_{\mathrm{liquidus}}\mathrm{,}$$

where $T_{\mathrm{solidus}}$ and $T_{\mathrm{liquidus}}$ are the wet solidus and dry liquidus temperatures of the considered rock.

From Table 21.2, we get:

## Dry Mantle model
$T_{\mathrm{solidus}} = \begin{cases} 1394 + 0.132899 \times P - 0.000005104 \times P^2 & \text{, if } P \leq 10000.0 \\2212 + 0.030819 \times (P - 10000.0) & \text{, if } P>10000.0\end{cases}$

$T_{\mathrm{liquidus}} = 2073 + 0.114 \times P$

## Wet Mantle model

$T_{\mathrm{solidus}} = \begin{cases} 1240 + \frac{49800}{P + 323} & \text{, if } P \leq 2400.0 \\1266 - 0.0118 \times P + 3.5 \times 10^{-6} \times P & \text{, if } P>2400.0\end{cases}$

$T_{\mathrm{liquidus}} = 2073 + 0.114 \times P$


In [12]:
# Path to the model directory

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT200_PT1292oC/1x1km2/NHK/RFT_Clc1_DT200_PT1292oC_lit80km_1x1km2_NHK' #ok
# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT200_PT1292oC/1x1km2/NHK/RFT_Clc10_DT200_PT1292oC_lit80km_1x1km2_NHK' #ok

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT290_PT1350oC/1x1km2/NHK/RFT_Clc1_DT290_PT1350oC_lit80km_1x1km2_NHK' #ok
# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT290_PT1350oC/1x1km2/NHK/RFT_Clc10_DT290_PT1350oC_lit80km_1x1km2_NHK' #ok

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT350_PT1397oC/1x1km2/NHK/RFT_Clc1_DT350_PT1397oC_lit80km_1x1km2_NHK' #ok
# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT350_PT1397oC/1x1km2/NHK/RFT_Clc10_DT350_PT1397oC_lit80km_1x1km2_NHK' #ok

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT400_PT1437oC/1x1km2/NHK/RFT_Clc1_DT400_PT1437oC_lit80km_1x1km2_NHK' #ok
# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit80km/DT400_PT1437oC/1x1km2/NHK/RFT_Clc10_DT400_PT1437oC_lit80km_1x1km2_NHK' #ok

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit150km/DT200_PT1352oC/1x1km2/NHK/RTF_Clc1_DT200_PT1352oC_lit150km_1x1km2_NHK' #ok
# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit150km/DT200_PT1352oC/1x1km2/NHK/RTF_Clc10_DT200_PT1352oC_lit150km_1x1km2_NHK' #ok

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit150km/DT290_PT1421oC/1x1km2/NHK/RFT_Clc1_DT290_PT1421oC_lit150km_1x1km2_NHK' #ok
# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit150km/DT290_PT1421oC/1x1km2/NHK/RFT_Clc10_DT290_PT1421oC_lit150km_1x1km2_NHK' #ok

# scenario = '/Doutorado/cenarios/mandyoc/rifting/lit150km/DT350_PT1460oC/1x1km2/NHK/RTF_Clc1_DT350_PT1460oC_lit150km_1x1km2_NHK' #ok
scenario = '/Doutorado/cenarios/mandyoc/rifting/lit150km/DT350_PT1460oC/1x1km2/NHK/RTF_Clc10_DT350_PT1460oC_lit150km_1x1km2_NHK' #ok

# scenario = '/RFT_Clc1_DT200_PT1292oC_lit80km_1x1km2_NHK' #just for testing

dataset = ['pressure',
		   'temperature',
           'strain',
           'density']

# local = True
local = False
if(local==True):
    model_path = machine_path + scenario
else:
    external_media = 'Joao_Macedo'
    if(path[1] == 'home'):
        model_path = f'/media/joao_macedo/{external_media}{scenario}'
    elif(path[1] == 'Users'):
        model_path = f'/Volumes/{external_media}{scenario}'

# model_path = machine_path + scenario
dataset = read_datasets(model_path, dataset)

Nx = int(dataset.nx)
Nz = int(dataset.nz)
Lx = float(dataset.lx)
Lz = float(dataset.lz)

dx = Lx/(Nx-1)
dz = Lz/(Nz-1)

t0 = dataset.time[0]
t1 = dataset.time[1]
dt = int(t1 - t0)

# start = 49
# end = 50
# step = 1

start = int(t0)
end = int(dataset.time.size - 1)
step = 1#5

#Dictionary to preserv the temporal order
Vmelt_dry_evolution = pymp.shared.dict() 
Vmelt_wet_evolution = pymp.shared.dict()

with pymp.Parallel() as p:
    for i in p.range(start, end+step, step):
        melt_dry = _calc_melt_dry(dataset.temperature[i], dataset.pressure[i])
        Vmelt_dry = np.sum(melt_dry)*dx*dz
         
        melt_wet = _calc_melt_wet(dataset.temperature[i], dataset.pressure[i])
        Vmelt_wet = np.sum(melt_wet)*dx*dz

        Vmelt_dry_evolution[i] = Vmelt_dry #coloca lista inteira na ordem certa para aquele i
        Vmelt_wet_evolution[i] = Vmelt_wet
            
########################################################################################
#Creating dataset of melt volume evolution

Vmelt_dry_dict = dict(Vmelt_dry_evolution) #converting to normal dictionary
Vmelt_wet_dict = dict(Vmelt_wet_evolution) #converting to normal dictionary

evol_dry = [] #empty list to concatenated
evol_wet = []

# Sorting dictionary to the order before paralelism
for i, j in zip(sorted(Vmelt_dry_dict), sorted(Vmelt_wet_dict)):
    evol_dry.append(Vmelt_dry_dict[i])
    evol_wet.append(Vmelt_wet_dict[j]) 
    
evol_dry = np.asarray(evol_dry)
evol_wet = np.asarray(evol_wet)

data_vars = {}
data_vars[f"vmelt_dry"] = evol_dry
data_vars[f"vmelt_wet"] = evol_wet
    
coords = {'time': dataset.time[start:end+step:step],
          'step': dataset.step[start:end+step:step]}

Vmelt_dataset = xr.Dataset(data_vars = data_vars,
                               coords = coords,
                               attrs=dict(description='Melt Volume evolution', units='%*km*km')
                              )
filename = f"_melt_volume_evolution_dataset.nc"
Vmelt_dataset.to_netcdf(f"{model_path}/{filename}")

dataset

<xarray.Dataset>
Dimensions:      (x: 1001, z: 301, time: 133)
Coordinates:
  * x            (x) float64 0.0 1e+03 2e+03 3e+03 ... 9.98e+05 9.99e+05 1e+06
  * z            (z) float64 -3e+05 -2.99e+05 -2.98e+05 ... -2e+03 -1e+03 0.0
  * time         (time) float64 0.0 0.5 1.0 1.5 2.0 ... 39.17 39.37 39.59 39.83
    step         (time) int32 0 100 200 300 400 ... 12900 13000 13100 13200
Data variables:
    pressure     (time, x, z) float64 ...
    temperature  (time, x, z) float64 ...
    strain       (time, x, z) float64 ...
    density      (time, x, z) float64 ...
Attributes: (12/94)
    nx:                                1001
    nz:                                301
    lx:                                1000000.0
    lz:                                300000.0
    multigrid:                         1
    solver:                            direct
    ...                                ...
    temperature_units:                 C
    density_units:                     kg/m^3
    heat_units:                        W/m^3
    viscosity_units:                   Pa s
    strain_rate_units:                 s^(-1)
    pressure_units:                    Pa